# Dog, cat image classification problem set

* 이번 학습에서는 처음부터 끝까지 Dog, cat dataset에 대한 분류 model을 구현합니다

### [CUDA](http://pytorch.org/docs/stable/cuda.html)

* cuda를 이용하겠습니다


In [1]:
import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
#train_on_gpu = False

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


---
## Load the [Data](http://pytorch.org/docs/stable/torchvision/datasets.html)

* 아미지를 다운로드 받습니다
* 폴더별로
 - test
 - train
 - validation

 data를 받습니다. 


In [2]:
!pip install googledrivedownloader

In [3]:
from os.path import exists
from google_drive_downloader import GoogleDriveDownloader as gdd
import tarfile 

#if exists("./Cat_Dog_data.tgz"):
#    !rm -rf ./Cat_Dog_data.tgz

gdd.download_file_from_google_drive(file_id='1WpY0qpe7yF5C5M52z1BMIzYVpDYiU3OV',
                                    dest_path = './Cat_Dog_data.tgz')

tf = tarfile.open("Cat_Dog_data.tgz")
tf.extractall()

## Problem 1 [20 points]: 

* Training, validation, test를 위한 dataloader, transform을 적절하게 준비해주세요
* 아래 data 준비하는 코딩을 수행하고, 아래 markdown에 준비한 과정 및 이유를 구체적으로 설명하세요
* 아래 답안 작성에 data의 구조에 대해서 설명하세요
* 코드에는 주석을 달아주세요.

In [4]:
# Coding

import torch
import numpy as np
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

# batch size
batch_size = 10

# 이미지 전처리
train_transforms = transforms.Compose([transforms.RandomRotation(30), # 랜덤 각도 회전
                                       transforms.RandomResizedCrop(224), # 랜덤 리사이즈 크롭
                                       transforms.RandomHorizontalFlip(), # 랜덤으로 수평 뒤집기
                                       transforms.ToTensor(), # 이미지를 텐서로
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]) # RGB 모든 픽셀을 0.5로 나눈 뒤 0.5로 뺌
                                                                                                
test_transforms = transforms.Compose([transforms.Resize(255), # 이미지 리사이즈
                                      transforms.CenterCrop(224), # 중앙 224 x 244 크롭
                                      transforms.ToTensor(), 
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]) # 이미지를 텐서로

# training, validation, testset 로드
trainset = ImageFolder(root='./train', transform=train_transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
                                                              
validset = ImageFolder(root='./validation', transform=train_transforms)
validloader = torch.utils.data.DataLoader(validset, batch_size=batch_size, shuffle=True)

# 테스트 셋은 셔플 안함
testset = ImageFolder(root='./test', transform=test_transforms)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size)

# cat dog 클래스 분류
classes = ['cat', 'dog']

print(len(trainset))
print(len(testset))
print(len(validset))

20000
2500
2500


**분석 및 설명:**

배치 사이즈는 10으로 한 뒤, 성능 향상을 위해 training transforms에서 랜덤 각도회전, 랜덤 리사이즈 크롭, 랜덤 뒤집기를 한다. 
그리고 test transfrom은 테스트셋이기 때문에 리사이즈와 크롭, 텐서만 적용시킨다

그 후에 training set, validation set, test set을 로드시킨 후 cat dog 클래스로 분류한다.



In [5]:
'''
############## 데이터 로드 테스트용
import matplotlib.pyplot as plt
%matplotlib inline

def imshow(img):
    img = img * 0.5 + 0.5 
    plt.imshow(np.transpose(img, (1, 2, 0)))  

dataiter = iter(trainloader) 
images, labels = dataiter.next()
images = images.numpy() 

fig = plt.figure(figsize=(25, 4))

for idx in np.arange(20): 
    ax = fig.add_subplot(2, 10, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(classes[labels[idx]])

print(images.shape)
'''

'\n############## 데이터 로드 테스트용\nimport matplotlib.pyplot as plt\n%matplotlib inline\n\ndef imshow(img):\n    img = img * 0.5 + 0.5 \n    plt.imshow(np.transpose(img, (1, 2, 0)))  \n\ndataiter = iter(trainloader) \nimages, labels = dataiter.next()\nimages = images.numpy() \n\nfig = plt.figure(figsize=(25, 4))\n\nfor idx in np.arange(20): \n    ax = fig.add_subplot(2, 10, idx+1, xticks=[], yticks=[])\n    imshow(images[idx])\n    ax.set_title(classes[labels[idx]])\n\nprint(images.shape)\n'

## Problem 2 [20 point]: Define the Network Architecture

* 구현하고자하는 network을 작성해주세요
* 아래 구현 방법과 이유를 구체적으로 설명하세요
* 코드에는 주석을 달아주세요. 
* 아래 모델을 구체적으로 설명하고, 설정 이유를 작성해주세요
* 본 과정에서는 직접 network을 구현하고, transfer learning은 활용하지 않도록 하겠습니다.

In [6]:
# 코드 작성
import torch.nn as nn
import torch.nn.functional as F

# CNN 작성
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()

      # input image = 224 x 244 x 3

      # 224 x 224 x 3 --> 112 x 112 x 32 maxpool
      self.conv1 = nn.Conv2d(3, 32, 3, padding=1) 
      # 112 x 112x 32 --> 56 x 56 x 64 maxpool
      self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
      # 56 x 56 x 64 --> 28 x 28 x 128 maxpool
      self.conv3 = nn.Conv2d(64, 128, 3, padding=1)     
      # 28 x 28 x 128 --> 14 x 14 x 256 maxpool
      self.conv4 = nn.Conv2d(128, 256, 3, padding=1)    

      # maxpool 2 x 2
      self.pool = nn.MaxPool2d(2, 2)
      
      # 28 x 28 x 128 vector flat 512개
      self.fc1 = nn.Linear(256 * 14 * 14, 512)
      # 카테고리 2개 클래스
      self.fc2 = nn.Linear(512, 2) 
      
      # dropout 적용
      self.dropout = nn.Dropout(0.5) # 0.25 해보고 0.5로 해보기. 값 저장하고나서

    def forward(self, x):
      # conv1 레이어에 relu 후 maxpool. 112 x 112 x 32
      x = self.pool(F.relu(self.conv1(x)))
      # conv2 레이어에 relu 후 maxpool. 56 x 56 x 64
      x = self.pool(F.relu(self.conv2(x)))
      # conv3 레이어에 relu 후 maxpool. 28 x 28 x 128
      x = self.pool(F.relu(self.conv3(x)))
      # conv4 레이어에 relu 후 maxpool. 14 x 14 x 256
      x = self.pool(F.relu(self.conv4(x)))

      # 이미지 펴기
      x = x.view(-1, 256 * 14 * 14) 
      # dropout 적용
      x = self.dropout(x)
      # fc 레이어에 삽입 후 relu
      x = F.relu(self.fc1(x))
      # dropout 적용
      x = self.dropout(x)
      
      # 마지막 logsoftmax 적용
      x = F.log_softmax(self.fc2(x), dim=1)
      return x

model = Net() # 모델 생성
print(model) # 출력

if train_on_gpu:
    model.cuda() # cuda 사용


Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


**분석 및 설명:**

정규화를 한 이미지 사이즈인 224 x 224 x 3 을 삽입한다. maxpool을 한 뒤 컨볼루션 3 x 3 x 3을 적용시키고, conv1에서 32개의 아웃풋을 적용하였기 때문에 112 x 112 x 32으로 첫번째 계산이 된다.
그 후에 conv2에서 동일하게 64개의 아웃풋을 적용한 후 maxpool 적용시켰기 때문에 56 x 56 x 64으로 계산이 되고, conv3에서 128개의 아웃풋, maxpool 적용시키면  28 x 28 x 128으로 계산이 되고, 마지막으로 conv4에서 14 x 14 x 256의 아웃풋이 나온다.

그리고 이미지를 편 후 오버피팅 방지를 위해 드롭아웃을 시킨다. 그리고 히든 레이어에 삽입 후 relu를 적용시킨 뒤 동일하게 드롭아웃 후 log_softmax를 적용시킨다. 만약 loss함수에서 CrossEntropyLoss를 사용 할 경우에는 log_softmax를 사용하지 않는다.

마지막으로 모델을 생성한 후 gpu를 사용한다

## Problem 3 [5 point]: Specify Loss Function

* Loss 함수와 optimizer를 구현하세요
* 선정 이유를 설명하세요


In [0]:
# 코드
import torch.optim as optim

criterion = nn.NLLLoss()

optimizer = optim.SGD(model.parameters(), lr=0.01)

 **설명:** loss 함수에서 NLLLoss와 CrossEntropyLoss 중 CrossEntropyLoss는 네트워크에서 log_softmax를 사용하지 않을때 사용하는 함수라고 하여 NLLLoss를 선택하였다.
 
optimizer는 SGD, Adam을 사용해보았는데 성능 검증 단계에서 Adam은 로스율이 0.69으로 고정되어 성능이 70%정도가 나왔다.
SGD는 정상적으로 로스율이 감소되어 성능이 80%이상이 나와 SGD를 사용하였다.


## Problem 4 [30 point]: Train the Network

* training loss와 validation loss를 기록하며 training을 구현하세요
* 만약 validation loss가 최소화된 모델을 저장하세요
* 코드에는 모두 주석을 포함해주세요
* training과정을 설명하고, training 결과를 분석해주세요

In [8]:
# 코드 작성

# epochs 30
n_epochs = 30

valid_loss_min = np.Inf

for epoch in range(1, n_epochs+1):
    # train, valid loss
    train_loss = 0.0
    valid_loss = 0.0
    
    # 모델 트레이닝
    model.train()
    # training set
    for data, target in trainloader:
        # cuda 사용
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()

        # 역전파 실행 전 gradient 0 초기화
        optimizer.zero_grad()
        # 모델 계산 후 output 저장
        output = model(data)
        # 로스율 계산
        loss = criterion(output, target)
        # 가중치 계산
        loss.backward()
        # 모델 parameter 업데이트
        optimizer.step()
        # 트레이닝 로스 계산
        train_loss += loss.item()*data.size(0)
        
    # validation 모델
    model.eval()
    validation_iter = iter(validloader)
    for data, target in validation_iter:
        # cuda 사용
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # 모델 계산 후 output 저장
        output = model(data)
        # 로스율 계산
        loss = criterion(output, target)
        # validation 로스율 계산
        valid_loss += loss.item()*data.size(0)
    
    # 평균 로스율
    train_loss = train_loss/len(trainloader.sampler)
    valid_loss = valid_loss/len(validloader.sampler)
        
    # training set, validation set 로스율 출력
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # 로스율이 낮아지면 model_catdog.pt에 저장
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_catdog.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 0.672942 	Validation Loss: 0.651699
Validation loss decreased (inf --> 0.651699).  Saving model ...
Epoch: 2 	Training Loss: 0.635230 	Validation Loss: 0.600129
Validation loss decreased (0.651699 --> 0.600129).  Saving model ...
Epoch: 3 	Training Loss: 0.602986 	Validation Loss: 0.571484
Validation loss decreased (0.600129 --> 0.571484).  Saving model ...
Epoch: 4 	Training Loss: 0.578175 	Validation Loss: 0.553068
Validation loss decreased (0.571484 --> 0.553068).  Saving model ...
Epoch: 5 	Training Loss: 0.566229 	Validation Loss: 0.539351
Validation loss decreased (0.553068 --> 0.539351).  Saving model ...
Epoch: 6 	Training Loss: 0.549421 	Validation Loss: 0.542284
Epoch: 7 	Training Loss: 0.537756 	Validation Loss: 0.524534
Validation loss decreased (0.539351 --> 0.524534).  Saving model ...
Epoch: 8 	Training Loss: 0.525244 	Validation Loss: 0.514252
Validation loss decreased (0.524534 --> 0.514252).  Saving model ...
Epoch: 9 	Training Loss: 0.518789 

**분석 및 설명:** 

epochs는 30으로 설정한다.

트레이닝 모델을 불러온 뒤 gradient를 0으로 초기화시킨다. 그 후에 모델링을 하고 로스율을 계산하고 가중치를 계산한다.
그리고 모델의 parameter를 업데이트 한 후 마지막 트레이닝 로스를 계산한다.

완료가 되면 validation 모델을 불러온 뒤 로스를 계산한다.

validation까지 완료가 되면 training set과 validation set의 평균 로스율을 계산하여 출력한다.

만약 validation 로스율이 기존보다 낮아진다면 model_catdog.pt에 저장시킨다.

출력 결과를 분석하면 정상적으로 Training Loss와 Validation Loss가 줄어드는 모습을 볼 수 있으며 가끔씩 Loss가 올라 저장이 안되는 경우도 발생하지만 정상적으로 Loss가 감소되었다.


## Problem 5 [5 point] Validation Loss가 최소화된 Model가져오기

* 최소 validation loss를 갖는 model을 불러옵니다

In [9]:
# 코드 작성

# 최소 로스율 모델 로딩
model.load_state_dict(torch.load('model_catdog.pt'))

<All keys matched successfully>

---
## Problem 6 [20point]: Test the Trained Network

* Test set을 활용하여 성능 검증
* Accuracy (분류 성공률)와 test loss를 출력하세요
* 코드에는 주석을 달아주세요
* 아래 test 결과에 대해서 간단하게 설명/분석 해주세요

In [11]:
# 코드 작성

# cat dog이기 때문에 2개 클래스
test_loss = 0.0
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))

model.eval()

for data, target in testloader:
    # cuda 사용
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # 데이터를 output에 삽입
    output = model(data)
    # loss율 계산
    loss = criterion(output, target)
    # loss율 업데이트
    test_loss += loss.item()*data.size(0)
    # 1차원, 정답률 확인
    _, pred = torch.max(output, 1)    
    # pred와 데이터를 비교한다
    correct_tensor = pred.eq(target.data.view_as(pred))
    # torrect_tensor를 numpy로 바꾼 뒤 gpu 계산 또는 cpu 계산
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # 몇 개 맞췄나 계산
    for i in range(batch_size): # 배치 사이즈로
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

        # batch_size 32 넣으면 index 오류가 발생함. 만약 배치사이즈 32넣을때 강제로 1250장씩 분류하면 중지되도록 설정
        # 평소에는 주석
        #if class_total == [1250.0, 1250.0]:
          #break

# 로스율 평균 계산
test_loss = test_loss/len(testloader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(2):
    # 각 클래스 별 확률 출력
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

# 최종 확률 출력
print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.216353

Test Accuracy of   cat: 91% (1148/1250)
Test Accuracy of   dog: 89% (1123/1250)

Test Accuracy (Overall): 90% (2271/2500)


**분석 및 설명:**
 
클래스가 cat, dog 2개의 클래스이기 때문에 class_correct와 class_total의 크기를 2으로 지정한다.

gpu를 사용하여 테스트 셋을 검증한다. Loss율을 계산하고 torch.max를 통해 정답률을 확인한다. 그리고 pred와 데이터를 비교하여 for문에서 배치 사이즈만큼 정답률을 계산한다.

그런데 배치사이즈를 32으로 설정했을 때 인덱스 범위 오류가 발생한다. 그래서 배치사이즈 32으로 테스트 할 때 강제로 class_total이 각각 1250장을 분류하면 break 되도록 설정해두었다. 주석처리

마지막으로 테스트셋의 로스율을 계산한 후 출력하며, 각 cat, dog 클래스에 대한 정답 확률을 출력하고 마지막으로 최종 확률을 출력한다.

---
## Problem 7: 전체적인 총평

* Data준비, Training과 validation 과정에서 성능 개선을 위해서 수행한 과정과 성공/실패 이유를 분석해주세요


**설명:**

1. batch_size 32 / Normalize 0.5 / 컨볼루션 3개 / Dropout 0.5 / Optimizer SGD 0.03 적용했을 때 

Test Loss: 0.542767

Test Accuracy of   cat: 86% (1080/1250)

Test Accuracy of   dog: 58% (733/1250)

Test Accuracy (Overall): 72% (1813/2500)

.

2. batch_size 32 / Normalize mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) / 컨볼루션 3개 / Dropout 0.25 / Optimizer SGD 0.01 적용했을 때 

Test Loss: 0.382337

Test Accuracy of   cat: 77% (964/1250)

Test Accuracy of   dog: 88% (1112/1250)

Test Accuracy (Overall): 83% (2076/2500)

.

3. batch_size 32 / Normalize mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) / 컨볼루션 3개 / Dropout 0.5 / Optimizer SGD 0.01 적용했을 때 

Test Loss: 0.392692

Test Accuracy of   cat: 77% (970/1250)

Test Accuracy of   dog: 87% (1093/1250)

Test Accuracy (Overall): 82% (2063/2500)

.

4. batch_size 10 / Normalize mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) / 컨볼루션 4개 / Dropout 0.5 / Optimizer SGD 0.01 적용했을 때 

Test Loss: 0.216353

Test Accuracy of   cat: 91% (1148/1250)

Test Accuracy of   dog: 89% (1123/1250)

Test Accuracy (Overall): 90% (2271/2500)

.

데이터 셋 준비 단계에서 평균과 표준편차를 직접 계산하려 했지만 계산하는 방법을 찾기 어려워 가장 많이 쓰는 std와 mean을(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 추가하였다. 그리고 배치사이즈를 10, 20, 32, 64를 적용시켜봤는데 10이 성능이 90%으로 가장 좋았다.

트레이닝 부분에서는 처음에 드롭아웃을 0.25으로 적용시켜 트레이닝 하였다. 그 후에 드롭아웃을 0.5으로 변경시켜 트레이닝 해보니 0.5으로 설정한 결과보다 0.25의 결과값이 1% 더 높았다.

optimizer를 SGD와 Adam을 사용해보았는데, SGD는 트레이닝 하는 과정에서 로스율이 0.3 후반까지 정상적으로 떨어지는 반면에 Adam 로스율이 0.69으로 거의 고정되어 더 이상 낮아지지 않았다. 

그래서 batch size가 32로 설정한 후 Adam의 결과는 78%의 결과가 나왔고, SGD를 적용했을 때에는 정상적으로 로스율이 낮아지면서 분류 확률이 83%가 나왔다. 구글에 검색을 해 보았을때는 SGD와 Adam중 하나를 고르라고 하면 성능이 좋은 Adam으로 고르라고 나와있지만 직접 트레이닝을 해 보니 정 반대의 결과가 나타났다. (참고 사이트 : http://www.gisdeveloper.co.kr/?p=8443)

많은 테스트 중 성능이 좋게 나온 모델 4가지를 선정하였다. cnn을 구현해보니 배치 사이즈와 정규화 그리고 optimizer를 수정하면 성능이 가장 큰 폭으로 변화하였다. 이미지를 정규화할 때 직접 이미지를 계산하면 더 성능이 좋아질 수 있었겠지만 방법을 찾지 못해 가장 보편적으로 많이 쓴 값을 넣은것이 아쉬운점이다.

따라서 여러 테스트 중 가장 성능이 높게 나온 모델은 성공률이 90%인 batch_size 10 / Normalize mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) / 컨볼루션 4개 / Dropout 0.5 / Optimizer 0.01 적용했을 때 이다.